In [10]:
import pickle

for data in ['bank77', 'clinic', 'goemo' , 'massive_I', 'massive_D']:
    results_dict = {}
    for emb_type in ['distilbert', 'sbert', 'openai', 'e5-large']:
        # Open the pickle file in read-binary mode
        with open("sims_offline_results_" + emb_type + "_" + data + ".pkl", "rb") as file:
            cur_results = pickle.load(file)
        results_dict[emb_type] = cur_results[emb_type]
        
    with open("results_" + data + ".pkl", "wb") as f:
        pickle.dump(results_dict, f)


In [107]:
import numpy as np
import pickle
results_summary = {}

for data in ['bank77', 'clinic', 'goemo' , 'massive_I', 'massive_D']:
    results_summary[data] = {}
    
    with open("results_" + data + ".pkl", "rb") as file:
        results_dict = pickle.load(file)

    for emb_type in ['distilbert', 'openai','e5-large', 'sbert']:
        results_summary[data][emb_type] = {}
       
        for i in ['kmeans','kmedoids']:
            acc = []
            nmi = []
            cen = []
            for seed in range(10):
                cur_results = results_dict[emb_type][seed][i]['results']
                acc.append(cur_results[0])
                nmi.append(cur_results[1])
                cen.append(cur_results[2])
                
            results_summary[data][emb_type][i] = list(np.round([100*np.mean(acc), 100*np.mean(nmi)],1)) + [np.round(np.mean(cen),3)]
    
        for force in [0, 10]:
            for niter in [1,5]:
                acc = []
                nmi = []
                cen = []
                for seed in range(10):
                    cur_results = results_dict[emb_type][seed][force][niter]['results']
                    acc.append(cur_results[0])
                    nmi.append(cur_results[1])
                    cen.append(cur_results[2])
                results_summary[data][emb_type][str(force) + '-' + str(niter)] = list(np.round([100*np.mean(acc), 100*np.mean(nmi)],1)) + [np.round(np.mean(cen),3)]

In [108]:
line = {}
for emb_type in ['distilbert', 'openai','e5-large', 'sbert']:
    line[emb_type] = {}
    
for data in ['bank77', 'clinic', 'goemo' , 'massive_I', 'massive_D']:

    for emb_type in ['distilbert', 'openai','e5-large', 'sbert']:

        acc = []
        nmi = []
        cen = []
        method_names = []
        for method in ['kmeans','kmedoids','0-1','0-5','10-1','10-5']:
            cur_results = results_summary[data][emb_type][method]
            acc.append(cur_results[0])
            nmi.append(cur_results[1])
            cen.append(cur_results[2])
            if method == '10-1':
                cur_method = 'k-LLMmeans FS 1-iter'
            elif method == '10-5':
                cur_method = 'k-LLMmeans FS 5-iter'
            elif method == '0-1':
                cur_method = 'k-LLMmeans 1-iter'
            elif method == '0-5':
                cur_method = 'k-LLMmeans 5-iter'
            elif method == 'kmeans':
                cur_method = 'k-means'
            elif method == 'kmedoids':
                cur_method = 'k-medoids'
            method_names.append(cur_method)

        # Find the maximum value
        val = max(acc)
        acc = [f"{{\\bf {x}}}" if x == val else x for x in acc]

        val = max(nmi)
        nmi = [f"{{\\bf {x}}}" if x == val else x for x in nmi]

        val = min(cen)
        cen = [f"{{\\bf {x}}}" if x == val else x for x in cen]
        
        for ii in range(len(method_names)):
            cur_method = method_names[ii]
            if data == 'bank77':
                if cur_method == 'k-means':
                    line[emb_type][cur_method] = '\\multirow{6}{*}{\\rotatebox{90}{' + emb_type + '}}' + '&' + cur_method + '&' + str(nmi[ii]) + '&' + str(cen[ii])# + '&' + str(cen[ii])
                else:
                    line[emb_type][cur_method] = '&' + cur_method + '&' + str(nmi[ii]) + '&' + str(cen[ii])# + '&' + str(cen[ii])
            else:
                line[emb_type][cur_method] = line[emb_type][cur_method] + '&&' + str(nmi[ii]) + '&' + str(cen[ii])# + '&' + str(cen[ii])

text = ""
for emb_type in ['distilbert', 'openai','e5-large', 'sbert']:
    text = text + '\\hline\n'
    for ii in range(len(method_names)):
        cur_method = method_names[ii]
        text = text + line[emb_type][cur_method] + '\\\\\n'
print(text)          

\hline
\multirow{6}{*}{\rotatebox{90}{distilbert}}&k-means&64.4&{\bf 0.335}&&77.2&{\bf 0.34}&&18.3&0.364&&58.1&0.366&&45.1&0.309\\
&k-medoids&56.0&0.591&&66.8&0.69&&14.0&0.769&&42.6&0.847&&27.2&0.81\\
&k-LLMmeans 1-iter&64.8&0.351&&77.4&0.352&&18.3&0.358&&58.6&{\bf 0.362}&&46.0&{\bf 0.294}\\
&k-LLMmeans 5-iter&64.8&0.358&&77.7&0.356&&18.3&0.354&&{\bf 59.0}&0.363&&45.6&0.306\\
&k-LLMmeans FS 1-iter&64.8&0.352&&78.1&0.346&&{\bf 18.9}&0.357&&58.8&0.365&&45.7&0.299\\
&k-LLMmeans FS 5-iter&{\bf 64.9}&0.363&&{\bf 78.7}&0.343&&18.8&{\bf 0.351}&&{\bf 59.0}&{\bf 0.362}&&{\bf 46.4}&0.295\\
\hline
\multirow{6}{*}{\rotatebox{90}{openai}}&k-means&83.0&0.225&&92.0&0.2&&20.5&0.287&&72.4&0.32&&67.9&0.246\\
&k-medoids&69.5&0.639&&77.7&0.694&&15.9&0.852&&52.4&0.86&&38.8&0.797\\
&k-LLMmeans 1-iter&83.6&0.221&&92.5&0.194&&{\bf 22.3}&0.278&&73.0&0.314&&69.6&0.232\\
&k-LLMmeans 5-iter&83.6&0.226&&92.8&0.186&&22.1&{\bf 0.275}&&73.5&0.307&&69.5&0.238\\
&k-LLMmeans FS 1-iter&83.8&{\bf 0.219}&&92.8&0.186&&21.9&